In [1]:
import pandas as pd
import geopandas as gpd
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import requests
import re
from geopandas.tools import sjoin
import rtree
import plotnine as p9
from plotnine import *
import numpy as np

# get census tract shapes
 

In [2]:
main_url = 'https://www2.census.gov/geo/tiger/TIGER2014/TRACT/'

raw_content = requests.get(main_url).text
parsed_content = BeautifulSoup(raw_content, 'html.parser')

find_links = parsed_content.findAll("a", attrs = {"href": re.compile("tract")})
zip_names = [link.get('href') for link in find_links]
zip_links = [main_url + link for link in zip_names]

In [3]:
list_of_tract_shapes = [gpd.read_file(link) for link in zip_links]
tract_shapes = pd.concat(list_of_tract_shapes)
tract_shapes.head()

,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,01,089,001902,01089001902,19.02,Census Tract 19.02,G5020,S,12784312,8256,+34.6825950,-086.5407538,"POLYGON ((-86.55607 34.67424, -86.55598 34.674..."
1,01,089,010800,01089010800,108,Census Tract 108,G5020,S,62434260,418101,+34.7623828,-086.4951024,"POLYGON ((-86.55461 34.78647, -86.55461 34.786..."
2,01,089,010701,01089010701,107.01,Census Tract 107.01,G5020,S,88171204,277431,+34.8667659,-086.5784822,"POLYGON ((-86.64287 34.87641, -86.64282 34.876..."
3,01,089,011014,01089011014,110.14,Census Tract 110.14,G5020,S,10927369,11607,+34.7176388,-086.7667595,"POLYGON ((-86.78654 34.70279, -86.78653 34.703..."
4,01,089,010301,01089010301,103.01,Census Tract 103.01,G5020,S,87926545,152312,+34.9392227,-086.5836942,"POLYGON ((-86.64212 34.93313, -86.64210 34.934..."


# get intersections of h2a violations and tracts


In [4]:
h2a = pd.read_pickle("../../qss20_finalproj_rawdata/summerwork/intermediate/h2a_combined_2014-2021_geocoded.pkl")
print(h2a.columns)
h2a_geom = gpd.GeoDataFrame(h2a, geometry=gpd.points_from_xy(h2a.geo_long, h2a.geo_lat))
h2a_geom.head()
h2a_geom

Index(['CASE_NUMBER', 'ATTORNEY_AGENT_CITY', 'JOB_TITLE', 'EMPLOYER_STATE',
       'SOC_CODE', 'data_source', 'REQUESTED_END_DATE_OF_NEED', 'SOC_TITLE',
       'EMPLOYER_CITY', 'JOB_END_DATE', 'CASE_STATUS', 'ATTORNEY_AGENT_STATE',
       'EMPLOYER_ADDRESS1', 'JOB_START_DATE', 'REQUESTED_START_DATE_OF_NEED',
       'EMPLOYER_POSTAL_CODE', 'EMPLOYER_NAME', 'WORKSITE_STATE',
       'ATTORNEY_AGENT_NAME', 'WORKSITE_CITY', 'DECISION_DATE',
       'EMPLOYER_FULLADDRESS', 'geo_lat', 'geo_long', 'geo_accuracy',
       'geo_accuracy_type'],
      dtype='object')


,CASE_NUMBER,ATTORNEY_AGENT_CITY,JOB_TITLE,EMPLOYER_STATE,SOC_CODE,data_source,REQUESTED_END_DATE_OF_NEED,SOC_TITLE,EMPLOYER_CITY,JOB_END_DATE,...,WORKSITE_STATE,ATTORNEY_AGENT_NAME,WORKSITE_CITY,DECISION_DATE,EMPLOYER_FULLADDRESS,geo_lat,geo_long,geo_accuracy,geo_accuracy_type,geometry
0,H-300-12345-457400,CRAIG,Livestock Worker,WY,45-2093,file_2014,2014-12-01 00:00:00,"Farmworkers, Farm, Ranch, and Aquacultural Ani...",Baggs,2014-12-01,...,WY,KANDANCE KROPINAK,Baggs,2014-03-03 10:30:27.000000,"PO Box 369, Baggs WY 82321",41.025579,-107.569976,0.33,place,POINT (-107.56998 41.02558)
1,H-300-12348-698598,CRAIG,Livestock Worker,WY,45-2093,file_2014,2014-12-15 00:00:00,"Farmworkers, Farm, Ranch, and Aquacultural Ani...",Baggs,2014-12-15,...,WY,KANDANCE KROPINAK,Baggs,2014-02-27 13:25:08.999996,"PO Box 6, Baggs WY 82321",41.025579,-107.569976,0.33,place,POINT (-107.56998 41.02558)
2,H-300-13007-934412,HEYBURN,Farm/Irrigation/Livestock Worker,ID,45-2092,file_2014,2014-11-15 00:00:00,"Farmworkers and Laborers, Crop, Nursery, and",Hammett,2014-11-15,...,ID,MICHAELENE ROWE,Hammett,2014-02-24 10:05:12.000002,"51854 State Hwy.78, Hammett ID 83627",42.945730,-115.466190,0.33,place,POINT (-115.46619 42.94573)
3,H-300-13009-524994,JOHNS ISLAND,Farmworker,GA,45-2092,file_2014,2014-06-20 00:00:00,"Farmworkers and Laborers, Crop, Nursery, and",Riceboro,2014-06-20,...,GA,MICHAEL LALICH,Riceboro,2014-03-28 12:47:28.999997,"3982 Jones Road NW, Riceboro GA 31323",31.631219,-81.478047,0.60,street_center,POINT (-81.47805 31.63122)
4,H-300-13050-453923,LEXINGTON,FARM WORKER FARM & RANCH ANIMALS,TX,45-2093,file_2014,2014-12-01 00:00:00,"Farmworkers, Farm, Ranch, and Aquacultural Ani...",MIDLAND,2014-12-01,...,TX,DONNA CARPENTER,MIDLAND,2014-01-24 09:12:22.000003,"600 NORTH BIG SPRING STREET, MIDLAND TX 79701",32.002923,-102.077871,1.00,rooftop,POINT (-102.07787 32.00292)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10261,H-300-21077-157874,Fairview,Ag Equipment Operator,MO,45-2091.00,file_2021,2021-12-01 00:00:00,Agricultural Equipment Operators,Bertrand,2021-12-01,...,MO,Kamron Martens,East Prairie,2021-03-30 00:00:00.000000,"2988 N Hwy B, Bertrand MO 63823",36.909500,-89.452580,0.33,place,POINT (-89.45258 36.90950)
10262,H-300-21084-174710,Charleston,Farm Laborer,SC,45-2092.02,file_2021,2021-12-01 00:00:00,"Farmworkers and Laborers, Crop",Orangeburg,2021-12-01,...,SC,Michael Todd Lalich,Orangeburg,2021-03-30 00:00:00.000000,"362 Jumbo Road, Orangeburg SC 29115",33.487988,-80.748377,1.00,rooftop,POINT (-80.74838 33.48799)
10263,H-300-20339-938887,Sevierville,Agricultural Equipment Operator,KS,45-2091.00,file_2021,2021-11-20 00:00:00,Agricultural Equipment Operators,Abilene,2021-11-20,...,KS,Heleen H van Tonder,Abilene,2021-03-31 00:00:00.000000,"306 3100 Ave, Abilene KS 67410",39.044791,-97.333413,1.00,rooftop,POINT (-97.33341 39.04479)
10264,H-300-20357-976186,Lake Park,Greenhouse Laborers,AL,45-2092.00,file_2021,2021-07-31 00:00:00,"Farmworkers and Laborers, Crop, Nursery, and G...",Union Springs,2021-07-31,...,OH,Chelsea A. Ibarra,Homeworth,2021-03-31 00:00:00.000000,"1727 Highway 223, Union Springs AL 36089",32.086960,-85.760985,1.00,rooftop,POINT (-85.76099 32.08696)


In [36]:
## filter out NA coordinates
nan_h2a_geom=h2a_geom.loc[pd.isnull(h2a_geom[['geo_lat', 'geo_long']]).any(axis=1)]
filtered_h2a_geom=h2a_geom.loc[~pd.isnull(h2a_geom[['geo_lat', 'geo_long']]).any(axis=1)]


In [37]:
h2a_tract_joined = sjoin(filtered_h2a_geom.set_crs(tract_shapes.crs), tract_shapes, how = 'left', op = 'intersects')

In [40]:
## rowbind the NA data to the merged data & Write pkl
final_tract_joined = h2a_tract_joined.append(nan_h2a_geom)
# final_tract_joined.shape
final_tract_joined.to_pickle("h2a_tract_intersections.pkl")

# map visual

In [51]:

cases_by_tract = final_tract_joined.groupby('GEOID').CASE_NUMBER.agg('nunique').reset_index()
merged = gpd.GeoDataFrame(cases_by_tract.merge(final_tract_joined[['GEOID', 'STATEFP', 'geometry']], 
                          how = 'right', on = 'GEOID').drop_duplicates(), geometry = 'geometry')
merged.sort_values('CASE_NUMBER')

In [49]:

plot = merged
plot['CASE_NUMBER'] = plot['CASE_NUMBER'].replace(0, np.nan)

In [52]:

p9.options.figure_size = (10, 8)

ggplot(plot) + \
    geom_map(aes(fill = 'CASE_NUMBER'), size = 0) + \
    scale_fill_gradient(low='yellow',high='red', na_value = 'grey') + \
    xlim(-130, -65) + \
    ylim(24, 50)

In [47]:
final_tract_joined[final_tract_joined.GEOID == '16083001100']


,CASE_NUMBER,ATTORNEY_AGENT_CITY,JOB_TITLE,EMPLOYER_STATE,SOC_CODE,data_source,REQUESTED_END_DATE_OF_NEED,SOC_TITLE,EMPLOYER_CITY,JOB_END_DATE,...,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON
328,H-300-15281-724016,NaN,SHEEPHERDER,ID,45-2093,file_2016,2016-12-09 00:00:00,"Farmworkers, Farm, Ranch, and Aquacultural Ani...",TWIN FALLS,NaT,...,001100,16083001100,11,Census Tract 11,G5020,S,2214343.0,0.0,+42.5568530,-114.4693499
796,H-300-15315-939576,NaN,Sheepherder,ID,45-2093,file_2016,2017-01-09 00:00:00,"Farmworkers, Farm, Ranch, and Aquacultural Ani...",TWIN FALLS,NaT,...,001100,16083001100,11,Census Tract 11,G5020,S,2214343.0,0.0,+42.5568530,-114.4693499
1148,H-300-15329-757634,NaN,Sheepherder,ID,45-2093,file_2016,2017-01-07 00:00:00,"Farmworkers, Farm, Ranch, and Aquacultural Ani...",TWIN FALLS,NaT,...,001100,16083001100,11,Census Tract 11,G5020,S,2214343.0,0.0,+42.5568530,-114.4693499
1514,H-300-15345-568879,NaN,SHEEPHERDER,ID,45-2093,file_2016,2017-02-07 00:00:00,"Farmworkers, Farm, Ranch, and Aquacultural Ani...",TWIN FALLS,NaT,...,001100,16083001100,11,Census Tract 11,G5020,S,2214343.0,0.0,+42.5568530,-114.4693499
3026,H-300-16014-170843,NaN,Sheepherder,ID,45-2093,file_2016,2017-03-08 00:00:00,"Farmworkers, Farm, Ranch, and Aquacultural Ani...",TWIN FALLS,NaT,...,001100,16083001100,11,Census Tract 11,G5020,S,2214343.0,0.0,+42.5568530,-114.4693499
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10092,H-300-20287-876832,NaN,RANGE WINTER SHEEPHERDER,ID,45-2093.00,file_2021,2021-03-31 00:00:00,"Farmworkers, Farm, Ranch, and Aquacultural Ani...",TWIN FALLS,2021-03-31,...,001100,16083001100,11,Census Tract 11,G5020,S,2214343.0,0.0,+42.5568530,-114.4693499
10129,H-300-20317-910084,NaN,RANGE SHEEPHERDER,ID,45-2093.00,file_2021,2021-10-01 00:00:00,"Farmworkers, Farm, Ranch, and Aquacultural Ani...",TWIN FALLS,2021-10-01,...,001100,16083001100,11,Census Tract 11,G5020,S,2214343.0,0.0,+42.5568530,-114.4693499
10133,H-300-20318-911634,NaN,RANGE WINTER SHEEPHERDER,ID,45-2093.00,file_2021,2021-05-31 00:00:00,"Farmworkers, Farm, Ranch, and Aquacultural Ani...",TWIN FALLS,2021-05-31,...,001100,16083001100,11,Census Tract 11,G5020,S,2214343.0,0.0,+42.5568530,-114.4693499
10184,H-300-21012-007690,NaN,RANGE SHEEPHERDER,ID,45-2093.00,file_2021,2021-10-31 00:00:00,"Farmworkers, Farm, Ranch, and Aquacultural Ani...",TWIN FALLS,2021-10-31,...,001100,16083001100,11,Census Tract 11,G5020,S,2214343.0,0.0,+42.5568530,-114.4693499
